In [ ]:
!pip install opendatasets --upgrade --quiet
!pip install kaggle --quiet

# Import required libraries
import opendatasets as od
import os
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.models import Model
import tensorflow as tf
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report
import time

# Download dataset from Kaggle
dataset_url = 'https://www.kaggle.com/datasets/jiayuanchengala/aid-scene-classification-datasets'
od.download(dataset_url)

# Set the path to the image folder
data_path = 'aid-scene-classification-datasets/AID'

# Load and preprocess dataset
image_data = []
labels = []

for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.endswith(".jpg"):
            image = cv2.imread(os.path.join(root, file))
            image = cv2.resize(image, (32, 32))
            image = img_to_array(image)
            image_data.append(image)
            label = root.split(os.path.sep)[-1]
            labels.append(label)

image_data = np.array(image_data, dtype="float") / 255.0
labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

x_train, x_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)
print(f'Training samples: {len(x_train)}, Testing samples: {len(x_test)}')

# Define Teacher Model
def teacher_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    return Model(inputs, outputs)

# Define Student Model
def student_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    return Model(inputs, outputs)

input_shape = (32, 32, 3)
num_classes = len(lb.classes_)

teacher = teacher_model(input_shape, num_classes)
student = student_model(input_shape, num_classes)

# Learning rate scheduler
def lr_scheduler(epoch, lr):
    return float(lr * tf.math.exp(-0.1)) if epoch >= 10 else float(lr)

# Train Teacher
teacher.compile(optimizer=SGD(learning_rate=1e-2, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
teacher.fit(x_train, y_train, batch_size=32, epochs=15, validation_data=(x_test, y_test), callbacks=[LearningRateScheduler(lr_scheduler)])


Skipping, found downloaded files in "./aid-scene-classification-datasets" (use force=True to force download)
Training samples: 8000, Testing samples: 2000
Epoch 1/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 93s 368ms/step - accuracy: 0.0524 - loss: 3.3484 - val_accuracy: 0.1415 - val_loss: 2.8607 - learning_rate: 0.0100
Epoch 2/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 90s 362ms/step - accuracy: 0.1525 - loss: 2.8764 - val_accuracy: 0.3230 - val_loss: 2.2472 - learning_rate: 0.0100
Epoch 3/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 91s 363ms/step - accuracy: 0.2809 - loss: 2.3660 - val_accuracy: 0.3690 - val_loss: 2.0696 - learning_rate: 0.0100
Epoch 4/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 142s 365ms/step - accuracy: 0.3710 - loss: 2.0622 - val_accuracy: 0.4245 - val_loss: 1.8960 - learning_rate: 0.0100
Epoch 5/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 90s 362ms/step - accuracy: 0.4366 - loss: 1.8263 - val_accuracy: 0.4700 - val_loss: 1.7461 - learning_rate: 0.0100
Epoch 6/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 147s 381ms/step - accuracy: 0

IndexError: list index out of range

In [ ]:

# Teacher Evaluation
teacher_score = teacher.evaluate(x_test, y_test, verbose=0)
print(f'Teacher test accuracy: {teacher_score[1] * 100:.2f}%')

# Teacher predictions (for distillation)
teacher_preds = teacher.predict(x_train)

# Distillation Loss Function with Nuclear Norm (alpha=0.25)
def distillation_loss(y_true, y_pred, teacher_preds, temperature=3.0, alpha=0.25):
    teacher_batch_preds = tf.gather(teacher_preds, tf.range(tf.shape(y_pred)[0]))
    y_true_soft = tf.keras.activations.softmax(y_true / temperature)
    y_pred_soft = tf.keras.activations.softmax(y_pred / temperature)
    teacher_soft = tf.keras.activations.softmax(teacher_batch_preds / temperature)
    soft_loss = tf.keras.losses.categorical_crossentropy(teacher_soft, y_pred_soft)
    hard_loss = tf.keras.losses.categorical_crossentropy(y_true_soft, y_pred_soft)

    first_layer_weights = None
    for layer in student.layers:
      if layer.weights:
        first_layer_weights = layer.weights[0]
        break
      if first_layer_weights is not None:
        singular_values = tf.linalg.svd(first_layer_weights, compute_uv=False)
        nuclear_norm = tf.reduce_sum(singular_values)
      else:
        nuclear_norm = 0.
    return alpha * (soft_loss + 0.001 * nuclear_norm) + (1. - alpha) * hard_loss

# Train Student
student.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=lambda y_true, y_pred: distillation_loss(y_true, y_pred, teacher_preds),
    metrics=['accuracy']
)
student.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_test, y_test), callbacks=[LearningRateScheduler(lr_scheduler)])

# Save trained models
teacher.save("teacher_model_1.h5")
student.save("student_model_1.h5")
print("Models saved successfully.")

# Evaluate Student on Clean Test Set
start_time = time.time()
y_pred_probs = student.predict(x_test)
inference_time = time.time() - start_time

y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

print("\n=== Final Student Model Evaluation on Clean Test Set ===")
print(classification_report(y_true, y_pred, target_names=lb.classes_))
print(f"Inference Time: {inference_time:.4f} seconds")

# -----------------------------
# Robustness Evaluation Against Gaussian Noise
# -----------------------------
y_reference_probs_1 = student.predict(x_test)
y_reference_1 = np.argmax(y_reference_probs_1, axis=1)

stddev_values = np.linspace(0, 1, 20)
noisy_accuracies, noisy_f1_scores, noisy_precisions, noisy_recalls = [], [], [], []
y_reference_list_1 = []
y_reference_probs_list_1 = []
y_pred_noisy_probs_list_1 = []
y_pred_noisy_list_1 = []

for stddev in stddev_values:
    noise = np.random.normal(loc=0.0, scale=stddev, size=x_test.shape)
    x_noisy = np.clip(x_test + noise, 0., 1.)

    y_pred_noisy_probs_1 = student.predict(x_noisy)
    y_pred_noisy_1 = np.argmax(y_pred_noisy_probs_1, axis=1)

    acc = accuracy_score(y_reference_1, y_pred_noisy_1)
    f1 = f1_score(y_reference_1, y_pred_noisy_1, average='weighted')
    precision = precision_score(y_reference_1, y_pred_noisy_1, average='weighted')
    recall = recall_score(y_reference_1, y_pred_noisy_1, average='weighted')

    noisy_accuracies.append(acc)
    noisy_f1_scores.append(f1)
    noisy_precisions.append(precision)
    noisy_recalls.append(recall)

    y_reference_list_1.append(y_reference_1)
    y_reference_probs_list_1.append(y_reference_probs_1)
    y_pred_noisy_probs_list_1.append(y_pred_noisy_probs_1)
    y_pred_noisy_list_1.append(y_pred_noisy_1)

    print(f"Noise STDDEV = {stddev:.2f} -> Accuracy = {acc:.4f}, F1 = {f1:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}")

Teacher test accuracy: 61.45%
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 84ms/step
Epoch 1/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 24s 87ms/step - accuracy: 0.0534 - loss: 3.4012 - val_accuracy: 0.1080 - val_loss: 3.4011 - learning_rate: 0.0010
Epoch 2/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 93ms/step - accuracy: 0.1666 - loss: 3.4011 - val_accuracy: 0.3320 - val_loss: 3.4010 - learning_rate: 0.0010
Epoch 3/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 41s 91ms/step - accuracy: 0.3056 - loss: 3.4010 - val_accuracy: 0.4050 - val_loss: 3.4009 - learning_rate: 0.0010
Epoch 4/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 39s 82ms/step - accuracy: 0.3943 - loss: 3.4009 - val_accuracy: 0.4365 - val_loss: 3.4008 - learning_rate: 0.0010
Epoch 5/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 42s 87ms/step - accuracy: 0.4356 - loss: 3.4008 - val_accuracy: 0.4850 - val_loss: 3.4008 - learning_rate: 0.0010
Epoch 6/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 41s 88ms/step - accuracy: 0.4840 - loss: 3.4008 - val_accuracy: 0.5385 - val_loss: 3.4007 - learning_rate: 0.0010
Epo

Models saved successfully.
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step

=== Final Student Model Evaluation on Clean Test Set ===
                   precision    recall  f1-score   support

          Airport       0.51      0.52      0.52        77
         BareLand       0.81      0.86      0.84        59
    BaseballField       0.84      0.90      0.87        42
            Beach       0.94      0.90      0.92        80
           Bridge       0.82      0.81      0.82        63
           Center       0.73      0.56      0.63        54
           Church       0.69      0.65      0.67        52
       Commercial       0.54      0.60      0.57        88
 DenseResidential       0.68      0.77      0.72        86
           Desert       0.86      0.86      0.86        59
         Farmland       0.59      0.56      0.58        78
           Forest       0.92      0.88      0.90        40
       Industrial       0.51      0.44      0.47        85
           Meadow       0.88      0.85      0.86

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Noise STDDEV = 0.16 -> Accuracy = 0.3775, F1 = 0.3445, Precision = 0.6034, Recall = 0.3775
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Noise STDDEV = 0.95 -> Accuracy = 0.0670, F1 = 0.0311, Precision = 0.0881, Recall = 0.0670
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
Noise STDDEV = 1.00 -> Accuracy = 0.0585, F1 = 0.0292, Precision = 0.0811, Recall = 0.0585


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print(y_reference_list_1)
print(y_reference_probs_list_1)
print(y_pred_noisy_probs_list_1)
print(y_pred_noisy_list_1)

[array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3])]
[array([[1.3868333e-02, 1.2304543e-03, 6.9909479e-04, ..., 1.3507868e-02,
        4.2634685e-02, 1.8447974e-03],
       [1.3944368e-04, 2.2540200e-06, 1.0067987e-03, ..., 9.2454714e-01,
        8.1075108e-05, 4.8130602e-07],
       [5.1575

In [ ]:
import os
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report
import time
import tensorflow as tf
from keras.callbacks import LearningRateScheduler

# Load models from .h5 files without compiling
teacher_1 = tf.keras.models.load_model('teacher_model_1.h5', compile=False)
student_1 = tf.keras.models.load_model('student_model_1.h5', compile=False)

# Load and preprocess data
data_path = 'aid-scene-classification-datasets/AID'
image_data = []
labels = []

for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.endswith(".jpg"):
            image = cv2.imread(os.path.join(root, file))
            image = cv2.resize(image, (32, 32))
            image = img_to_array(image)
            image_data.append(image)
            label = root.split(os.path.sep)[-1]
            labels.append(label)

image_data = np.array(image_data, dtype="float") / 255.0
labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

x_train, x_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)

# Get teacher predictions for distillation
teacher_preds_1 = teacher_1.predict(x_train)

# Define distillation loss function with nuclear norm
def distillation_loss(y_true, y_pred, teacher_preds, temperature=3.0, alpha=0.25):
    teacher_batch_preds = tf.gather(teacher_preds, tf.range(tf.shape(y_pred)[0]))
    y_true_soft = tf.keras.activations.softmax(y_true / temperature)
    y_pred_soft = tf.keras.activations.softmax(y_pred / temperature)
    teacher_soft = tf.keras.activations.softmax(teacher_batch_preds / temperature)
    soft_loss = tf.keras.losses.categorical_crossentropy(teacher_soft, y_pred_soft)
    hard_loss = tf.keras.losses.categorical_crossentropy(y_true_soft, y_pred_soft)

    first_layer_weights = None
    for layer in student_1.layers:
        if layer.weights:
            first_layer_weights = layer.weights[0]
            break

    if first_layer_weights is not None:
        singular_values = tf.linalg.svd(first_layer_weights, compute_uv=False)
        nuclear_norm = tf.reduce_sum(singular_values)
    else:
        nuclear_norm = 0.

    return alpha * (soft_loss + 0.001 * nuclear_norm) + (1. - alpha) * hard_loss

def lr_scheduler(epoch, lr):
    return float(lr * tf.math.exp(-0.1)) if epoch >= 10 else float(lr)

# Compile student model with custom loss
student_1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=lambda y_true, y_pred: distillation_loss(y_true, y_pred, teacher_preds_1),
    metrics=['accuracy']
)

# Retrain student model (if needed)
student_1.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_test, y_test), callbacks=[LearningRateScheduler(lr_scheduler)])

# Evaluate student on clean test set
start_time = time.time()
y_pred_probs = student_1.predict(x_test)
inference_time = time.time() - start_time

y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

print("\n=== Final Student Model Evaluation on Clean Test Set ===")
print(classification_report(y_true, y_pred, target_names=lb.classes_))
print(f"Inference Time: {inference_time:.4f} seconds")

# Robustness evaluation
y_reference_probs_1 = student_1.predict(x_test)
y_reference_1 = np.argmax(y_reference_probs_1, axis=1)

stddev_values = np.linspace(0, 1, 20)

for stddev in stddev_values:
    noise = np.random.normal(loc=0.0, scale=stddev, size=x_test.shape)
    x_noisy = np.clip(x_test + noise, 0., 1.)

    y_pred_noisy_probs_1 = student_1.predict(x_noisy)
    y_pred_noisy_1 = np.argmax(y_pred_noisy_probs_1, axis=1)

    acc = accuracy_score(y_reference_1, y_pred_noisy_1)
    f1 = f1_score(y_reference_1, y_pred_noisy_1, average='weighted')
    precision = precision_score(y_reference_1, y_pred_noisy_1, average='weighted')
    recall = recall_score(y_reference_1, y_pred_noisy_1, average='weighted')

    print(f"Noise STDDEV = {stddev:.2f} -> Accuracy = {acc:.4f}, F1 = {f1:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}")


250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 85ms/step
Epoch 1/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 91ms/step - accuracy: 0.8157 - loss: 3.4037 - val_accuracy: 0.6115 - val_loss: 3.4023 - learning_rate: 0.0010
Epoch 2/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 41s 92ms/step - accuracy: 0.7717 - loss: 3.4018 - val_accuracy: 0.5715 - val_loss: 3.4014 - learning_rate: 0.0010
Epoch 3/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 40s 87ms/step - accuracy: 0.7128 - loss: 3.4011 - val_accuracy: 0.5710 - val_loss: 3.4011 - learning_rate: 0.0010
Epoch 4/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 22s 87ms/step - accuracy: 0.6605 - loss: 3.4010 - val_accuracy: 0.5530 - val_loss: 3.4010 - learning_rate: 0.0010
Epoch 5/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 42s 92ms/step - accuracy: 0.6790 - loss: 3.4008 - val_accuracy: 0.5755 - val_loss: 3.4009 - learning_rate: 0.0010
Epoch 6/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 41s 90ms/step - accuracy: 0.7010 - loss: 3.4008 - val_accuracy: 0.5010 - val_loss: 3.4010 - learning_rate: 0.0010
Epoch 7/30
250/250 ━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step
Noise STDDEV = 0.21 -> Accuracy = 0.2575, F1 = 0.2286, Precision = 0.4207, Recall = 0.2575


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step
Noise STDDEV = 0.26 -> Accuracy = 0.1855, F1 = 0.1552, Precision = 0.3040, Recall = 0.1855


...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
Noise STDDEV = 0.95 -> Accuracy = 0.0465, F1 = 0.0093, Precision = 0.0080, Recall = 0.0465


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step
Noise STDDEV = 1.00 -> Accuracy = 0.0475, F1 = 0.0117, Precision = 0.0530, Recall = 0.0475


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print(y_reference_list_1)
print(y_reference_probs_list_1)
print(y_pred_noisy_probs_list_1)
print(y_pred_noisy_list_1)

[array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3]), array([12, 27, 24, ..., 23, 15,  3])]
[array([[1.3868333e-02, 1.2304543e-03, 6.9909479e-04, ..., 1.3507868e-02,
        4.2634685e-02, 1.8447974e-03],
       [1.3944368e-04, 2.2540200e-06, 1.0067987e-03, ..., 9.2454714e-01,
        8.1075108e-05, 4.8130602e-07],
       [5.1575